# Korean Embedding 모델을 SageMaker 배포 및 추론
- 이 노트북은 SageMaker Notebook Instance 의 conday_pytorch_p39 에서 테스트 되었습니다.

Model Ref:
- BM-K/KoSimCSE-roberta
    - https://huggingface.co/BM-K/KoSimCSE-roberta
Inference Code Ref:    
- Huggingface Sagemaker-sdk - Deploy 🤗 Transformers for inference
    - https://github.com/huggingface/notebooks/blob/main/sagemaker/11_deploy_model_from_hf_hub/deploy_transformer_model_from_hf_hub.ipynb
- Sentence Embeddings with Hugging Face Transformers, Sentence Transformers and Amazon SageMaker - Custom Inference for creating document embeddings with Hugging Face's Transformers
    - https://github.com/huggingface/notebooks/blob/main/sagemaker/17_custom_inference_script/sagemaker-notebook.ipynb
    

# 1. HF Hub로 부터 모델 및 토큰 나이저 로딩

In [2]:
import torch
from transformers import AutoModel, AutoTokenizer

def cal_score(a, b):
    '''
    코사인 유사도 구하는 함수
    '''
    if len(a.shape) == 1: a = a.unsqueeze(0)
    if len(b.shape) == 1: b = b.unsqueeze(0)

    a_norm = a / a.norm(dim=1)[:, None]
    b_norm = b / b.norm(dim=1)[:, None]
    return torch.mm(a_norm, b_norm.transpose(0, 1)) * 100

model = AutoModel.from_pretrained('BM-K/KoSimCSE-roberta')
tokenizer = AutoTokenizer.from_pretrained('BM-K/KoSimCSE-roberta')



In [27]:
print("model.config.max_length: ", model.config.max_length)
print("tokenizer.max_len_single_sentence: ", tokenizer.max_len_single_sentence)
print("tokenizer.max_model_input_sizes: \n", tokenizer.max_model_input_sizes)
print("tokenizer.model_max_length: ", tokenizer.model_max_length)

model.config.max_length:  20
tokenizer.max_len_single_sentence:  510
tokenizer.max_model_input_sizes: 
 {'bert-base-uncased': 512, 'bert-large-uncased': 512, 'bert-base-cased': 512, 'bert-large-cased': 512, 'bert-base-multilingual-uncased': 512, 'bert-base-multilingual-cased': 512, 'bert-base-chinese': 512, 'bert-base-german-cased': 512, 'bert-large-uncased-whole-word-masking': 512, 'bert-large-cased-whole-word-masking': 512, 'bert-large-uncased-whole-word-masking-finetuned-squad': 512, 'bert-large-cased-whole-word-masking-finetuned-squad': 512, 'bert-base-cased-finetuned-mrpc': 512, 'bert-base-german-dbmdz-cased': 512, 'bert-base-german-dbmdz-uncased': 512, 'TurkuNLP/bert-base-finnish-cased-v1': 512, 'TurkuNLP/bert-base-finnish-uncased-v1': 512, 'wietsedv/bert-base-dutch-cased': 512}
tokenizer.model_max_length:  512


## Max Length 테스트

In [140]:
import numpy as np

def show_embedding(tokenizer, model, sentences):
    inputs = tokenizer(sentences, padding=True, truncation=True, return_tensors="pt")
    embeddings, _ = model(**inputs, return_dict=False)
    
    print("embeddings: ", embeddings.shape)
    return None

# sentences0 = [
#                 "아마존 매장에는 새로운 판매자를 위한 많은 기회가 있습니다. 판매할 수 있는 것은 제품, 범주 및 브랜드에 따라 다릅니다. 일부 범주는 모든 판매자에게 열려 있는 반면 다른 범주는 프로페셔널 셀러 계정이 필요합니다. 특정 제품은 판매 승인이 필요하고 다른 카테고리에는 타사 판매자가 판매할 수 없는 제품이 포함됩니다. 어떤 제품부터 시작해야 합니까? 가장 수익성이 좋은 틈새 시장은 무엇입니까? 아이디어가 필요하면 이 38가지 온라인 비즈니스 아이디어 목록을 확인하십시오. 아마존에서 시작한 판매자의 이야기를 듣게 될 것입니다. 전 세계 고객에게 다가갈 수 있도록 성장했습니다.\
#                 아마존 매장에는 새로운 판매자를 위한 많은 기회가 있습니다. 판매할 수 있는 것은 제품, 범주 및 브랜드에 따라 다릅니다. 일부 범주는 모든 판매자에게 열려 있는 반면 다른 범주는 프로페셔널 셀러 계정이 필요합니다. 특정 제품은 판매 승인이 필요하고 다른 카테고리에는 타사 판매자가 판매할 수 없는 제품이 포함됩니다. 어떤 제품부터 시작해야 합니까? 가장 수익성이 좋은 틈새 시장은 무엇입니까? 아이디어가 필요하면 이 38가지 온라인 비즈니스 아이디어 목록을 확인하십시오. 아마존에서 시작한 판매자의 이야기를 듣게 될 것입니다. 전 세계 고객에게 다가갈 수 있도록 성장했습니다\
#                 아마존 매장에는 새로운 판매자를 위한 많은 기회가 있습니다. 판매할 수 있는 것은 제품, 범주 및 브랜드에 따라 다릅니다. 일부 범주는 모든 판매자에게 열려 있는 반면 다른 범주는 프로페셔널 셀러 계정이 필요합니다. 특정 제품은 판매 승인이 필요하고 다른 카테고리에는 타사 판매자가 판매할 수 없는 제품이 포함됩니다. 어떤 제품부터 시작해야 합니까? 가장 수익성이 좋은 틈새 시장은 무엇입니까? 아이디어가 필요하면 이 38가지 온라인 비즈니스 아이디어 목록을 확인하십시오. 아마존에서 시작한 판매자의 이야기를 듣게 될 것입니다. 전 세계 고객에게 다가갈 수 있도록 성장했습니다\
#                 아마존 매장에는 새로운 판매자를 위한 많은 기회가 있습니다. 판매할 수 있는 것은 제품, 범주 및 브랜드에 따라 다릅니다. 일부 범주는 모든 판매자에게 열려 있는 반면 다른 범주는 프로페셔널 셀러 계정이 필요합니다. 특정 제품은 판매 승인이 필요하고 다른 카테고리에는 타사 판매자가 판매할 수 없는 제품이 포함됩니다. 어떤 제품부터 시작해야 합니까? 가장 수익성이 좋은 틈새 시장은 무엇입니까? 아이디어가 필요하면 이 38가지 온라인 비즈니스 아이디어 목록을 확인하십시오. 아마존에서 시작한 판매자의 이야기를 듣게 될 것입니다. 전 세계 고객에게 다가갈 수 있도록 성장했습니다\
#                 아마존 매장에는 새로운 판매자를 위한 많은 기회가 있습니다. 판매할 수 있는 것은 제품, 범주 및 브랜드에 따라 다릅니다. 일부 범주는 모든 판매자에게 열려 있는 반면 다른 범주는 프로페셔널 셀러 계정이 필요합니다. 특정 제품은 판매 승인이 필요하고 다른 카테고리에는 타사 판매자가 판매할 수 없는 제품이 포함됩니다. 어떤 제품부터 시작해야 합니까? 가장 수익성이 좋은 틈새 시장은 무엇입니까? 아이디어가 필요하면 이 38가지 온라인 비즈니스 아이디어 목록을 확인하십시오. 아마존에서 시작한 판매자의 이야기를 듣게 될 것입니다. 전 세계 고객에게 다가갈 수 있도록 성장했습니다\
#                 "
# ]

sentences0 = [
                "아마존 매장에는 새로운 판매자를 위한 많은 기회가 있습니다. 판매할 수 있는 것은 제품, 범주 및 브랜드에 따라 다릅니다. 일부 범주는 모든 판매자에게 열려 있는 반면 다른 범주는 프로페셔널 셀러 계정이 필요합니다. 특정 제품은 판매 승인이 필요하고 다른 카테고리에는 타사 판매자가 판매할 수 없는 제품이 포함됩니다. 어떤 제품부터 시작해야 합니까? 가장 수익성이 좋은 틈새 시장은 무엇입니까? 아이디어가 필요하면 이 38가지 온라인 비즈니스 아이디어 목록을 확인하십시오. 아마존에서 시작한 판매자의 이야기를 듣게 될 것입니다. 전 세계 고객에게 다가갈 수 있도록 성장했습니다.\
                아마존 매장에는 새로운 판매자를 위한 많은 기회가 있습니다. 판매할 수 있는 것은 제품, 범주 및 브랜드에 따라 다릅니다. 일부 범주는 모든 판매자에게 열려 있는 반면 다른 범주는 프로페셔널 셀러 계정이 필요합니다. 특정 제품은 판매 승인이 필요하고 다른 카테고리에는 타사 판매자가 판매할 수 없는 제품이 포함됩니다. 어떤 제품부터 시작해야 합니까? 가장 수익성이 좋은 틈새 시장은 무엇입니까? 아이디어가 필요하면 이 38가지 온라인 비즈니스 아이디어 목록을 확인하십시오. 아마존에서 시작한 판매자의 이야기를 듣게 될 것입니다. 전 세계 고객에게 다가갈 수 있도록 성장했습니다\
                아마존 매장에는 새로운 판매자를 위한 많은 기회가 있습니다. 판매할 수 있는 것은 제품, 범주 및 브랜드에 따라 다릅니다. 일부 범주는 모든 판매자에게 열려 있는 반면 다른 범주는 프로페셔널 셀러 계정이 필요합니다. 특정 제품은 판매 승인이 필요하고 다른 카테고리에는 타사 판매자가 판매할 수 없는 제품이 포함됩니다. 어떤 제품부터 시작해야 합니까? 가장 수익성이 좋은 틈새 시장은 무엇입니까? 아이디어가 필요하면 이 38가지 온라인 비즈니스 아이디어 목록을 확인하십시오. 아마존에서 시작한 판매자의 이야기를 듣게 될 것입니다. 전 세계 고객에게 다가갈 수 있도록 성장했습니다\
                아마존 매장에는 새로운 판매자를 위한 많은 기회가 있습니다.판매할 수 있는 것은 제품, 범주 및 브랜드에 따라 다릅니다.일부 범주는 모든 판매자에게 열려 있는 반면 다른 범주는 프로페셔널 셀러 계정이 필요합니다.특정 제품은 판매 승인이 필요하고 다른 카테고리에는 타사 판매자가 판매를 더욱더 할 수 있습니다. 이것을 추가 토큰 입니다\
                "
]


print("sentences0 : ", len(sentences0[0]))


sentences0 :  1217


In [141]:


show_embedding(tokenizer, model, sentences0)

embeddings:  torch.Size([1, 512, 768])


# 2. 세이지 메이커로 모델 배포

In [13]:
import sagemaker
import boto3

try:
    role = sagemaker.get_execution_role()
except ValueError:
    iam = boto3.client('iam')
    role = iam.get_role(RoleName='sagemaker_execution_role')['Role']['Arn']

print(f"sagemaker role arn: {role}")

sagemaker role arn: arn:aws:iam::057716757052:role/gen_ai_gsmoon


## HF Model ID, HF_TASK 정의

In [106]:
from sagemaker.huggingface import HuggingFaceModel

# Hub Model configuration. https://huggingface.co/models
hub = {
  'HF_MODEL_ID':'BM-K/KoSimCSE-roberta', # model_id from hf.co/models
  'HF_TASK':'feature-extraction',
  'SAGEMAKER_MODEL_SERVER_TIMEOUT':'3600', 
  'TS_MAX_RESPONSE_SIZE':'2000000000',
  'TS_MAX_REQUEST_SIZE':'2000000000',
  'MMS_MAX_RESPONSE_SIZE':'2000000000',
  'MMS_MAX_REQUEST_SIZE':'2000000000'
}        


# create Hugging Face Model Class
huggingface_model = HuggingFaceModel(
   env=hub,
   role=role, # iam role with permissions to create an Endpoint
   transformers_version="4.26", # transformers version used
   pytorch_version="1.13", # pytorch version used
   py_version="py39", # python version of the DLC
)

## 모델 배포

In [107]:
%%time

# deploy model to SageMaker Inference
predictor = huggingface_model.deploy(
   initial_instance_count=1,
   # instance_type="ml.m5.xlarge"
   instance_type="ml.g5.2xlarge"    
)

---------!CPU times: user 210 ms, sys: 7.7 ms, total: 218 ms
Wall time: 5min 2s


In [108]:
predictor.endpoint_name

'huggingface-pytorch-inference-2023-06-04-10-59-43-906'

# 3. 추론

In [142]:
import numpy as np

payload_0 = {
    "inputs" : sentences0
}



def predict_payload(data):
    res = predictor.predict(data=data)
    res = np.array(res) # .squeeze().squeeze()
    print("res: ", res.shape)
    # print("embedding dimension: ", len(res[0][0][0]))
    return None

predict_payload(payload_0)

res:  (1, 1, 517, 768)


# 4. 엔드포인트 삭제

In [143]:
# delete endpoint
predictor.delete_model()
predictor.delete_endpoint()